In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
df= pd.read_csv('qoute_dataset.csv')

In [5]:
df.head()

,quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


In [6]:
df.shape

(3038, 2)

In [9]:
quotes= df['quote']
quotes.head()

,quote
0,“The world as we have created it is a process ...
1,"“It is our choices, Harry, that show what we t..."
2,“There are only two ways to live your life. On...
3,"“The person, be it gentleman or lady, who has ..."
4,"“Imperfection is beauty, madness is genius and..."


In [10]:
quotes = quotes.str.lower()

In [12]:
import string
translator = str.maketrans('', '', string.punctuation)
quotes = quotes.apply(lambda x: x.translate(translator))

In [13]:
quotes.head()

,quote
0,“the world as we have created it is a process ...
1,“it is our choices harry that show what we tru...
2,“there are only two ways to live your life one...
3,“the person be it gentleman or lady who has no...
4,“imperfection is beauty madness is genius and ...


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [16]:
vocab_size = 8978
tokenizer =Tokenizer(num_words= vocab_size)
tokenizer.fit_on_texts(quotes)

In [18]:
word_index = tokenizer.word_index
print(len(word_index))
list(word_index.items())[:10]

8978


[('the', 1),
 ('you', 2),
 ('to', 3),
 ('and', 4),
 ('a', 5),
 ('i', 6),
 ('is', 7),
 ('of', 8),
 ('that', 9),
 ('it', 10)]

In [20]:
sequence = tokenizer.texts_to_sequences(quotes)

In [23]:
for i in range(3):
    print(quotes[i])
    print(sequence[i])

“the world as we have created it is a process of our thinking it cannot be changed without changing our thinking”
[713, 62, 29, 19, 16, 946, 10, 7, 5, 1156, 8, 70, 293, 10, 145, 12, 809, 104, 752, 70, 2461]
“it is our choices harry that show what we truly are far more than our abilities”
[947, 7, 70, 871, 373, 9, 433, 21, 19, 465, 14, 294, 52, 54, 70, 3676]
“there are only two ways to live your life one is as though nothing is a miracle the other is as though everything is a miracle”
[1337, 14, 53, 201, 714, 3, 81, 15, 36, 37, 7, 29, 329, 93, 7, 5, 1157, 1, 101, 7, 29, 329, 126, 7, 5, 3677]


In [24]:
X= []
y= []

for seq in sequence:
  for i in range(1, len(seq)):
    input_seq = seq[:i]
    output_seq = seq[i]
    X.append(input_seq)
    y.append(output_seq)

In [25]:
len(X)

85270

In [26]:
len(y)

85270

In [27]:
max_len = max(len(x) for x in X)
print(max_len)

745


In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
X_padded = pad_sequences(X, maxlen= max_len, padding='pre')

In [30]:
X_padded

array([[   0,    0,    0, ...,    0,    0,  713],
       [   0,    0,    0, ...,    0,  713,   62],
       [   0,    0,    0, ...,  713,   62,   29],
       ...,
       [   0,    0,    0, ...,    9,   19, 1125],
       [   0,    0,    0, ...,   19, 1125,    3],
       [   0,    0,    0, ..., 1125,    3,  169]], dtype=int32)

In [31]:
y= np.array(y)

In [32]:
y

array([ 62,  29,  19, ...,   3, 169, 101])

In [33]:
X_padded.shape

(85270, 745)

In [34]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(y, num_classes=vocab_size)

In [35]:
y.shape

(85270,)

In [36]:
y_one_hot.shape

(85270, 8978)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM, Dense

In [40]:
embedding_dim = 50
rnn_units = 128

In [42]:
rnn_model = Sequential()

rnn_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
rnn_model.add(SimpleRNN(units=rnn_units))
rnn_model.add(Dense(units=vocab_size, activation='softmax'))

In [43]:
rnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [45]:
rnn_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [46]:
lstm_model = Sequential()
lstm_model.add(
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len)
)
lstm_model.add(LSTM(units=rnn_units))
lstm_model.add(Dense(units=vocab_size, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [47]:
lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [48]:
epochs = 100
batch_size = 128

history_lstm = lstm_model.fit(
    X_padded,
    y_one_hot,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1
)

Epoch 1/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 37s 52ms/step - accuracy: 0.0338 - loss: 7.0489 - val_accuracy: 0.0428 - val_loss: 6.6824
Epoch 2/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.0533 - loss: 6.3590 - val_accuracy: 0.0678 - val_loss: 6.5492
Epoch 3/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 52ms/step - accuracy: 0.0771 - loss: 6.0650 - val_accuracy: 0.0873 - val_loss: 6.4337
Epoch 4/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.0931 - loss: 5.8393 - val_accuracy: 0.0946 - val_loss: 6.4064
Epoch 5/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 30s 51ms/step - accuracy: 0.1052 - loss: 5.6656 - val_accuracy: 0.1017 - val_loss: 6.4026
Epoch 6/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.1155 - loss: 5.4983 - val_accuracy: 0.1045 - val_loss: 6.4107
Epoch 7/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1253 - loss: 5.3274 - val_accuracy: 0.1077 - val_loss: 6.4209
Epoch 8/100
600/600 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.1346 - loss: 5

In [49]:
lstm_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, 745, 50)        │       448,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8978)           │     1,158,162 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,096,132 (19.44 MB)

 Trainable params: 1,698,710 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,397,422 (12.96 MB)

In [53]:
from tensorflow.keras.models import load_model

In [55]:
lstm_model.save("lstm_model.h5")

In [56]:
lstm_model = load_model("lstm_model.h5")

In [57]:
index_to_word = {}
for word, index in word_index.items():
    index_to_word[index] = word

In [58]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [59]:
def predictor(model,tokenizer,text,max_len):
  text = text.lower()

  seq = tokenizer.texts_to_sequences([text])[0]
  seq = pad_sequences([seq], maxlen=max_len, padding='pre')

  pred = model.predict(seq,verbose = 0)
  pred_index = np.argmax(pred)
  return index_to_word[pred_index]


In [61]:
seed_text = "what are you"
next_word = predictor(lstm_model,tokenizer,seed_text,max_len)
print(next_word)

worrying


In [69]:
seed_text = "life is "
next_word = predictor(lstm_model,tokenizer,seed_text,max_len)
print(next_word)

what


In [64]:
def generate_text(model,tokenizer,seed_text,max_len,n_words):
  for _ in range(n_words):
    next_word = predictor(model,tokenizer,seed_text,max_len)
    if next_word == "":
      break
    seed_text += " " + next_word
  return seed_text

In [66]:
seed = "are you a "
generate_text = generate_text(lstm_model,tokenizer,seed,max_len,10)
print(generate_text)

are you a  lot of success who may be a very time of


In [67]:
import pickle
with open("tokenizer.pkl", "wb") as f:
  pickle.dump(tokenizer, f)

In [68]:
with open("max_len.pkl", "wb") as f:
  pickle.dump(max_len, f)